### 任务三、 人脸识别模型微调

本案例在小样本数据集上微调InceptionResnetV1模型。

#### 0. 环境准备

In [2]:
!pip install -U openmim
!pip install facenet_pytorch==2.5.2
!mim install mmcv

Looking in indexes: https://pypi.virtaicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 112.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 217.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 65.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 130.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 247.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 145.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 218.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 229.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 147.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 204.5 MB/s eta 0:00:00
  Created wheel for oss2: filename=oss2-2.17.0-py3-none-any.whl size=112372 s

In [4]:
!cp $GEMINI_DATA_IN1/20180402-114759-vggface2.pt ~/.cache/torch/checkpoints

In [5]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-15 14:38:41.648989: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-15 14:38:43.531841: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 14:38:43.531868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 14:38:43.531891: E te

根据当前运行环境是否有GPU可用，设置device的值

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

在该设备上运行: cuda:0


#### 1. 定义训练超参数

<font color=red size=3>动手练习1</font> <br>
首先将facenet_data数据集挂载路径下 `test_images`目录拷贝到`/gemini/code`下。

在`<1>`处，设置`test_images`数据集所在的路径。

In [7]:
data_dir = './test_images'

batch_size = 32
epochs = 8

workers = 0 if os.name == 'nt' else 8

#### 2. 定义MTCNN模块

In [8]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### 3. 迭代获取裁剪后的人脸。

<font color=red size=3>动手练习2</font> <br>

在`<1>`处，使用`datasets.ImageFolder(）`方法，加载数据集，并且将图像的大小调整为512x512像素。

在`<2>`处，调用mtcnn模型，获取裁剪后的人脸图像，并保存到y指定的路径。

In [9]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))

# 修改了dataset对象的samples属性，这是一个列表，其中包含图像的路径和标签。
# 列表推导式遍历原始samples中的每个元素（p是图像路径，_是一个占位符，表示忽略标签）。
# 对于每个图像路径p，生成一个新的路径p.replace(data_dir, data_dir + '_cropped')，这个新路径在原始路径的基础上添加了'_cropped'后缀。
# 这样，原始图像和它们的裁剪版本都将包含在dataset中。
dataset.samples = [ (p, p.replace(data_dir, data_dir + '_cropped')) for p, _ in dataset.samples ]

# 创建了一个DataLoader对象，用于批量加载数据集。 
# batch_size是每个批次中的样本数量。
# collate_fn=training.collate_pil指定了一个自定义的批处理函数，该函数用于处理PIL图像。
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

# 遍历DataLoader对象生成的批次。
# i是批次的索引。
# (x, y)是当前批次中的图像和对应的路径。
for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\r第 {} 批，共 {} 批'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

第 1 批，共 1 批

#### 4. 定义Inception Resnet V1模块

In [10]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### 5. 创建优化器、调度器

<font color=red size=3>动手练习3</font> <br>

在`<1>`处，创建Adam优化器，传入resnet模型的所有参数，设置了优化器的学习率lr为0.001。

在`<2>`处，创建学习率调度器（scheduler），用于在训练过程中调整学习率。请采用MultiStepLR调度器，指定优化器是上面创建的优化器实例optimizer，指定在epoch为4和8处降低学习率，。

In [11]:

optimizer = optim.Adam(resnet.parameters(), lr=1e-3)
scheduler = MultiStepLR(optimizer, [4, 8])


#### 6. 创建数据加载器

<font color=red size=3>动手练习4</font> <br>

在`<1>`处，使用`transforms.Compose()`方法，将`np.float32`、`transforms.ToTensor()`、`fixed_image_standardization`三个变换方法组合成一个序列。

在`<2>`处，创建`datasets.ImageFolder`数据集对象，数据集文件夹的路径为之前保存截取人脸图像所保存的路径，并且对图像的进行trans所定义的一系列变换。

在`<3>`处，编写一行代码，打乱img_inds数组的顺序；

在`<4>`处，使用切片操作选取img_inds的前80%的数据作为训练集的索引。

在`<5>`处，使用切片操作选取img_inds的前80%的数据作为训练集的索引。

In [23]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization,
    transforms.Resize((160, 160), antialias=True)
])
dataset = datasets.ImageFolder(data_dir, transform=trans)

img_inds = np.arange(len(dataset))

n = len(img_inds)
shuffled_indices = torch.randperm(n)
shuffled_indices = torch.randperm(n)
split_indices = int(n*0.8)

train_inds = shuffled_indices[:split_indices]
val_inds = shuffled_indices[split_indices:]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### 7. 创建损失函数和评估函数

<font color=red size=3>动手练习5</font> <br>

在`<1>`处，创建一个交叉熵损失函数实例，直接使用PyTorch提供的函数。

在`<2>`处，创建一个包含评估指标键值对的字典，具体包含'fps' 和 'acc'两个键，它们的值分别是training.BatchTimer() 和 training.accuracy。


In [24]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy,
}

#### 8. 微调训练模型

此步骤执行训练过程，请执行代码。

In [ ]:
# 创建一个SummaryWriter实例，用于将训练或评估过程中的数据写入TensorBoard日志文件
writer = SummaryWriter()
# 设置SummaryWriter实例的iteration属性为0，表示当前的迭代次数或步骤数，设置interval属性为10，表示每10个迭代步骤，SummaryWriter会记录一次数据。
writer.iteration, writer.interval = 0, 10

print('\n\n初始化')
print('-' * 10)

# 将resnet模型设置为评估模式。
resnet.eval()

# 调用pass_epoch函数，传入resnet：正在评估的模型，loss_fn：损失函数，val_loader：提供验证集数据的DataLoader实例，
# batch_metrics=metrics：包含用于评估模型性能的指标，如准确率、每秒帧数等。
# show_running=True：指示pass_epoch函数在控制台显示训练或评估过程中的实时进度。
# device：指定模型和数据运行的设备（CPU或GPU）。
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

# 训练循环，用于训练和评估模型。
for epoch in range(epochs):
    print('\n循环 {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    # 将模型resnet设置为训练模式。
    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



初始化
----------
Valid |     1/1    | loss:    0.0101 | fps:    0.0169 | acc:    1.0000   

循环 1/8
----------
